# ALICE Masterclass R$_{AA}$

    Laden der erforderlichen Pakete

In [ ]:
import ALICE_RAA_Tools as at
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

Liste der verfügbaren Zentralitäten:

    0-5%, 0-10%, 5-10%, 10-20%, 20-30%, 30-40%, 40-50%, 50-60%, 60-70%, 70-80%, 80-90%

__ACHTUNG:__
Ihr darf nicht überall die vorhandene Zentralität geändert werden. Die Zentralitätsklasse 70-80% dient später als Referenz für R$_{cp}$ und darf nicht gelöscht werden.
    

In [ ]:
listCentralities = [(60,70), (70,80)]

    Erstellen eines Dictionaries

In [ ]:
dictCentralities = at.create_dictionary(listCentralities)


# 1. Event-Analyse

### 1.1 Einlesen der Events

    Zählen der Events

In [ ]:
event_lines = at.count_lines('event_information.csv')
print("Number of events: '" + str(event_lines) + "'!")

    Einlesen der Event-Information aus der CSV-Datei

In [ ]:
df_events = pd.read_csv("event_information.csv", header=None, names=['eventMult', 'eventCent'])

In [ ]:
print(df_events.head())

### 1.2 Bearbeiten der Daten

    Hinzufügen von Spalten zum DataFrame für jedes Zentralitätsintervall

In [ ]:
for key in dictCentralities:
    df_events[key] = False

print(df_events.head())

    Füllen der Zentralitätsinformation und Bestimmung der Anzahl an Events pro Zentralitätsintervall

In [ ]:
dictEventsCent = {}
for row in df_events.iterrows():
    keys = at.find_centralities(dictCentralities, row[1]['eventCent'])
    if keys:
        for cent in keys:
            df_events.loc[row[0], cent] = True
            if cent in dictEventsCent:
                dictEventsCent[cent] += 1
            else:
                dictEventsCent[cent] = 1
                
print(df_events.head())
print(dictEventsCent)

### 1.3 Darstellung der Ergebnisse

    Erzeugung & Darstellung der Histogramme zur Teilchenmultiplizität

In [ ]:
hTPC_0_100, bins = at.create_event_histos(df_events['eventMult'])
hTPC_60_70, bins = at.create_event_histos(df_events[df_events['60-70'] == True]['eventMult'])
hTPC_70_80, bins = at.create_event_histos(df_events[df_events['70-80'] == True]['eventMult'])

In [ ]:
hTPC_0_100_err = np.sqrt(hTPC_0_100)
hTPC_60_70_err = np.sqrt(hTPC_60_70)
hTPC_70_80_err = np.sqrt(hTPC_70_80)

In [ ]:
plt.errorbar(bins[:-1], hTPC_0_100, yerr=hTPC_0_100_err, fmt='g+', label='0-100%')
plt.errorbar(bins[:-1], hTPC_60_70, yerr=hTPC_60_70_err, fmt='b+', label='60-70%')
plt.errorbar(bins[:-1], hTPC_70_80, yerr=hTPC_70_80_err, fmt='r+', label='70-80%')
#zu den Einstllungen unter 'fmt' ist folgende Seite nützlich
#https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.plot.html#matplotlib.pyplot.plot

plt.xlabel('Number of TPC tracks')
plt.ylabel('Counts')
plt.ylim(1,2E4)
plt.yscale('log')

plt.legend(loc='upper right')
plt.savefig('TPC_tracks.pdf')

    2D-Plot: Teilchenspurmultiplizität gegen Zentralität

In [ ]:
plt.hist2d(df_events['eventMult'],df_events['eventCent'], bins=(200,100), range=[[0, 2000], [0, 100]], cmap=plt.cm.jet, norm=mpl.colors.LogNorm())

plt.colorbar()
plt.xlabel('Number of TPC tracks')
plt.ylabel('Centrality')
plt.savefig('TPC_tracks_2D.pdf')

# 2. Teilchenspuren und Impulsspektren

    Liste der verfügbaren Zentralitäten:
 
     0-5%, 0-10%, 5-10%, 10-20%, 20-30%, 30-40%, 40-50%, 50-60%, 60-70%, 70-80%, 80-90%

## 2.1 Einlesen der Pb-Pb Daten

In [ ]:
print("Number of tracks: '38042122'! ~600MB Datei (ungepackt)")

In [ ]:
dictMomCent = {}
for key in dictCentralities:
    dictMomCent[key] = []

In [ ]:
df_tracks = pd.read_pickle("./track_info.pkl", 'bz2').to_numpy()

### 2.2 Sortieren nach Zentralität

In [ ]:
for line in df_tracks:
    keys = at.find_centralities(dictCentralities, line[1])
    if keys:
        for key in keys:
            dictMomCent[key].append(line[0])

### 2.3 Erzeugen der Histogramme

In [ ]:
binsPt = at.get_bins()
binWidth = at.get_bin_width(binsPt)
x_bin_width = np.asarray(binWidth)/2

In [ ]:
hPbPb_60_70, _ = np.histogram(dictMomCent['60-70'], binsPt)
hPbPb_60_70_err = np.sqrt(hPbPb_60_70) / dictEventsCent['60-70']
hPbPb_60_70 = hPbPb_60_70 / binWidth
hPbPb_60_70 = hPbPb_60_70 / dictEventsCent['60-70']

hPbPb_70_80, _ = np.histogram(dictMomCent['70-80'], binsPt)
# Spezialfall für 70-80%, ein Bin ist Null - problematisch für spätere Division - daher 0 durch 1 ersetzen
hPbPb_70_80 = np.where(hPbPb_70_80==0,1,hPbPb_70_80)
hPbPb_70_80_err = np.sqrt(hPbPb_70_80) / dictEventsCent['70-80']
hPbPb_70_80 = hPbPb_70_80 / binWidth
hPbPb_70_80 = hPbPb_70_80 / dictEventsCent['70-80']

### 2.4 Einlesen der Proton-Proton Referenz

In [ ]:
pp = np.genfromtxt('pp_reference.dat')
print(pp)
pp_err = np.divide(pp,10) #Annahme: 10% Fehler auf der pp Messung

### 2.5 Darstellung der Transversalimpulsspektren geladener Teilchen

In [ ]:
plt.errorbar(binsPt[:-1], hPbPb_60_70, xerr=x_bin_width, yerr=hPbPb_60_70_err, fmt='g.', label='60-70%')
plt.errorbar(binsPt[:-1], hPbPb_70_80, xerr=x_bin_width, yerr=hPbPb_70_80_err, fmt='b.', label='70-80%')
plt.errorbar(binsPt[:-1], pp,          xerr=x_bin_width, yerr=pp_err,          fmt='y+', label='pp')

plt.xscale('log')
plt.yscale('log')
plt.xlim(0.1, 20)
plt.ylim(1E-6, 1E4)

plt.xlabel('$p_{T}$')
plt.ylabel('Counts')
plt.legend(loc='upper right')
plt.show()

### 2.6 Definition der Zentralitätsklassen

Definition der Anzahl an Kollisionen für einzelne Zentralitäten

In [ ]:
dictNColl = {'0-5': 1686.87,
             '0-10': 1502.7,
             '5-10': 1319.89,
             '10-20': 923.89,
             '20-30': 558.68,
             '30-40': 321.20,
             '40-50': 171.67,
             '50-60': 85.13,
             '60-70': 38.51,
             '70-80': 15.78,
             '80-90': 6.32,
             'pp':1.
            }

print(dictNColl)

### 2.7 Berechnung und Darstellung des R$_{\text{CP}}$

In [ ]:
RCP_60_70 = np.divide(hPbPb_60_70 / dictNColl['60-70'], hPbPb_70_80 / dictNColl['70-80'])                                                                

In [ ]:
RCP_60_70_err = at.fehlerberechnung(hPbPb_60_70, hPbPb_60_70_err, dictNColl['60-70'], hPbPb_70_80, hPbPb_70_80_err, dictNColl['70-80'])

In [ ]:
plt.errorbar(binsPt[:-1], RCP_60_70, xerr=x_bin_width, yerr=RCP_60_70_err, fmt='b.', label='60-70%')

plt.ylim(-0.1,2)
plt.xlabel('$p_{T}$')
plt.ylabel(r'$R_{cp}=\frac{1/N^{AA,X}_{evt}1/<N_{coll,X}>dN^{AA,X}/dp_t}{1/N^{PbPb,per}_{evt}1/<N_{coll,per}>dN^{PbPb,per}/dp_t}$')
plt.legend(loc='upper right')
plt.savefig('RCP_compared.pdf')

### 2.8 Berechnung und Darstellung des R$_\text{AA}$

In [ ]:
RAA_60_70 = np.divide(hPbPb_60_70 / dictNColl['60-70'], pp)
RAA_70_80 = np.divide(hPbPb_70_80 / dictNColl['70-80'], pp)

In [ ]:
RAA_60_70_err = at.fehlerberechnung(hPbPb_60_70, hPbPb_60_70_err, dictNColl['60-70'], pp, pp_err, dictNColl['pp'])
RAA_70_80_err = at.fehlerberechnung(hPbPb_70_80, hPbPb_70_80_err, dictNColl['70-80'], pp, pp_err, dictNColl['pp'])

In [ ]:
plt.errorbar(binsPt[:-1], RAA_60_70, xerr=x_bin_width, yerr=RAA_60_70_err, fmt='b.', label='60-70%')
plt.errorbar(binsPt[:-1], RAA_70_80, xerr=x_bin_width, yerr=RAA_70_80_err, fmt='g.', label='70-80%')

plt.ylim(0.01,1.2)
plt.xlabel('$p_{T}$')
plt.ylabel(r'$R_{AA}=\frac{1/N^{AA}_{evt}dN^{AA}/dp_t}{<N_{coll}>1/N_{pp_{evt}dN^{pp}/dp_t}}$')
plt.legend(loc='upper right')
plt.savefig('RAA_compared.pdf')